In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.ndimage import shift
from IPython.display import clear_output
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
import seaborn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
train_data = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/train.csv"))
test_data = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/test.csv"))
sample_submission = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv"))
train_data = train_data.drop("Id",axis=1)
test_data = test_data.drop("Id",axis=1)

In [4]:
X_train_data = train_data.drop("SalePrice",axis=1)
y_train_data = train_data["SalePrice"]

**Prepairing the data :**

In [5]:
imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(pd.concat([X_train_data,test_data]))

SimpleImputer(strategy='most_frequent')

In [6]:
c = X_train_data.columns
i_train = X_train_data.index
i_test = test_data.index
X_train_data = pd.DataFrame(imputer.transform(X_train_data),columns = c,index = i_train)
test_data = pd.DataFrame(imputer.transform(test_data),columns = c,index=i_test)

In [7]:
 str_cat = [ "MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope",
  "Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st",
  "Exterior2nd","MasVnrType","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
  "Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional","FireplaceQu","GarageType",
  "GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence","MiscFeature","SaleCondition","SaleType"]

In [8]:
def OneHotE(dataframe,cat_col,cat_encoder):
    poly_features = PolynomialFeatures(degree=2,include_bias=False)
    dataframe_num = dataframe.drop(cat_col,axis=1)
    dataframe_cat = dataframe[cat_col]
    
    col = dataframe_num.columns
    dataframe_num =  poly_features.fit_transform(dataframe_num)
    dataframe_num = pd.DataFrame(dataframe_num,index = dataframe.index , columns = poly_features.get_feature_names_out(col))
    
    scaler = StandardScaler()
    for column in dataframe_num.columns:
        dataframe_num[column] = scaler.fit_transform(np.array(dataframe_num[column]).reshape(-1,1))
    
    dataframe_cat = cat_encoder.transform(dataframe_cat)
    dataframe_cat = pd.DataFrame(dataframe_cat.toarray(),columns=cat_encoder.get_feature_names_out(),index=dataframe.index)
    dataframe = pd.concat([dataframe_num,dataframe_cat],axis=1)    
    return dataframe

In [9]:
cat_encoder = OneHotEncoder()

In [10]:
cat_encoder.fit(pd.concat([X_train_data[str_cat],test_data[str_cat]]))

OneHotEncoder()

In [11]:
X_train_data = OneHotE(X_train_data,str_cat,cat_encoder)
test_data = OneHotE(test_data,str_cat,cat_encoder)

**Training a model:**

In [12]:
pipeline = Pipeline([("classifier",LinearRegression())])

In [13]:
param_grid = [{"classifier__fit_intercept":[True,False]}] 

In [14]:
grid_search = GridSearchCV(pipeline,param_grid,cv=3,scoring="r2")

In [15]:
grid_search.fit(X_train_data,y_train_data)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('classifier', LinearRegression())]),
             param_grid=[{'classifier__fit_intercept': [True, False]}],
             scoring='r2')

In [16]:
grid_search.best_params_


{'classifier__fit_intercept': True}

In [17]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.440716,0.048726,0.029364,0.001615,True,{'classifier__fit_intercept': True},-809.375350,-64.671272,-214.583908,-362.876843,321.599271,1
1,0.369023,0.003173,0.028545,0.000651,False,{'classifier__fit_intercept': False},-801.401354,-64.673146,-228.249080,-364.774527,315.881319,2


**Prediction :**

In [18]:
final_evaluation = grid_search.predict(test_data)

In [19]:
d = {'Id':np.arange(len(final_evaluation)) + 1461,'SalePrice':final_evaluation}
df = pd.DataFrame(data=d)
df

,Id,SalePrice
0,1461,53717.955541
1,1462,149892.954930
2,1463,166442.511907
3,1464,181066.556205
4,1465,198314.350869
...,...,...
1454,2915,102401.533737
1455,2916,75187.430590
1456,2917,219738.024901
1457,2918,200454.133040


In [20]:
df.to_csv('out.csv',index=False)